# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [3]:
# Use this cell to write your code for Task 1
import pandas as pd
df = pd.read_csv('loyalty.csv')
df = df.fillna('Unknown')
df['first_month'].replace('.', 0, inplace=True)
df['first_month'] = df['first_month'].astype(float)
df['promotion'].replace(['NO', 'YES'], ['No', 'Yes'], inplace=True)
clean_data = df

# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [5]:
# Use this cell to write your code for Task 2
df1 = pd.DataFrame()
df1['avg_spend'] = df.groupby('loyalty_years').agg({'spend':'mean'}).round(2)
df1['var_spend'] = df.groupby('loyalty_years').agg({'spend':'var'}).round(2)
df1.reset_index(inplace=True)
spend_by_years = df1
print(spend_by_years)

  loyalty_years  avg_spend  var_spend
0           0-1     110.56       9.30
1           1-3     129.31       9.65
2           10+     117.41      16.72
3           3-5     124.55      11.09
4          5-10     135.15      14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [19]:
# Use this cell to write your code for Task 3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Prepare the data
X_train = train.drop(columns=['customer_id', 'spend'])
y_train = train['spend']
X_test = test.drop(columns=['customer_id'])

# Convert categorical variables to dummy variables
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Ensure the test set has the same dummy variables as the training set
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
test['spend'] = model.predict(X_test)

# Create the result dataframe
base_result = test[['customer_id', 'spend']]

In [25]:
# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train.drop(['customer_id', 'spend'], axis=1)
y = train['spend']

# Convert categorical variables to dummy variables
X = pd.get_dummies(X, drop_first=True)
y = pd.get_dummies(y, drop_first=True)

# Ensure the test set has the same dummy variables as the training set
X = X.reindex(columns=X.columns, fill_value=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
p = model.predict(X_test)

mse = mean_squared_error(y_test, p)
print("Mean Squared Error:", mse)

r2 = r2_score(y_test, p)
print("R-squared:", r2)

Mean Squared Error: 0.0013206891116976812
R-squared: -0.00029312334915966603


# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [9]:
# Use # Use this cell to write your code for Task 4
from sklearn.ensemble import GradientBoostingRegressor

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Prepare the data
X_train = train.drop(columns=['customer_id', 'spend'])
y_train = train['spend']
X_test = test.drop(columns=['customer_id'])

# Convert categorical variables to dummy variables
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Ensure the test set has the same dummy variables as the training set
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Train a Gradient Boosting model
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)

# Make prediction on the test data
test['spend'] = gb.predict(X_test)

compare_result = test[['customer_id', 'spend']]
compare_result.to_csv('compare_result.csv', index=False)